In [143]:
import re
import pandas as pd 
import numpy as np

In [488]:
Posts = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Posts.csv', \
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [609]:
Posts

,Id,PostTypeId,ParentID,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,CommunityOwnedDate,ClosedDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
0,48870881,1,NaN,48871194.0,2018-02-19T17:05:48.193,1,50.0,<p>I have a json file with data as below:</p>\...,2804169.0,2804169.0,NaN,2018-02-19T18:42:53.487,2018-02-19T18:53:19.357,NaN,NaN,PHP search array return all values,<php><arrays>,2.0,0,NaN
1,48870882,1,NaN,48870993.0,2018-02-19T17:05:49.097,0,27.0,<p>I am currently working on a web application...,5128923.0,2598770.0,NaN,2018-02-19T17:13:33.920,2018-02-19T17:13:49.300,NaN,NaN,Creating a reactive user input tab,<c#><html><asp.net><.net><visual-studio>,2.0,0,NaN
2,48870883,2,48869547.0,NaN,2018-02-19T17:05:50.073,1,NaN,<p>Try to add hint to the query:</p>\r\n\r\n<p...,8214734.0,8214734.0,NaN,2018-02-20T10:20:21.007,2018-02-20T10:20:21.007,NaN,NaN,NaN,NaN,NaN,2,NaN
3,48870884,2,3865445.0,NaN,2018-02-19T17:05:50.273,0,NaN,In support of Sean's post:\r\n\r\n<p>The place...,2055283.0,NaN,NaN,NaN,2018-02-19T17:05:50.273,NaN,NaN,NaN,NaN,NaN,0,NaN
4,48870885,2,48870539.0,NaN,2018-02-19T17:05:51.300,0,NaN,<p>I discovered the convention (couldn't find ...,1052870.0,NaN,NaN,NaN,2018-02-19T17:05:51.300,NaN,NaN,NaN,NaN,NaN,1,NaN
5,48870886,1,NaN,NaN,2018-02-19T17:05:52.677,0,27.0,<p>So I am trying to create records in mongodb...,8353588.0,NaN,NaN,NaN,2018-02-19T17:18:09.357,NaN,NaN,Mongoose Validation Error,<node.js><mongodb><express><mongoose><mongoose...,1.0,2,NaN
6,48870888,1,NaN,48871145.0,2018-02-19T17:06:01.170,1,17.0,<p>I am trying to find out a way how add a col...,9381432.0,NaN,NaN,NaN,2018-02-19T17:21:55.740,NaN,NaN,R - Calculating annual incrementals across dif...,<r>,1.0,0,NaN
7,48870889,2,48870709.0,NaN,2018-02-19T17:06:01.910,0,NaN,"<p>Group by the user, then aggregate a list of...",3182843.0,NaN,NaN,NaN,2018-02-19T17:06:01.910,NaN,NaN,NaN,NaN,NaN,0,NaN
8,48870890,1,NaN,NaN,2018-02-19T17:06:06.810,0,19.0,<p>I need a simple and clear example on how to...,3613597.0,3613597.0,NaN,2018-02-19T17:18:00.510,2018-02-19T17:18:00.510,NaN,2018-02-19T17:09:58.357,Passing props from parent component to a neste...,<javascript><reactjs><react-native>,0.0,0,NaN
9,48870891,2,48870253.0,NaN,2018-02-19T17:06:16.113,12,NaN,<p>With combination of <code>re.findall()</cod...,3185459.0,3185459.0,NaN,2018-02-19T17:13:55.730,2018-02-19T17:13:55.730,NaN,NaN,NaN,NaN,NaN,8,NaN


## the number of answers against tags

In [663]:
def answers_user(user_id):
    filtered = []
    pattern = '[<>]'
    regex = re.compile(pattern, flags = re.IGNORECASE)
    answers_id = list(Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]['ParentID'])

    for i in range(len(answers_id)):
        tags = Posts.loc[(Posts.Id == answers_id[i])]['Tags']
        try:
            filtered.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_answers = pd.DataFrame({'Tag':filtered}).groupby('Tag').size()
    return freq_answers

In [664]:
#example
answers_user(476)

Tag
abstraction            1
access                 1
angular                1
api                    1
backend                1
character-encoding     2
design-patterns        1
encoding               1
ftp                    1
int                    1
interface              1
iso-8859-1             1
javascript             2
mixins                 1
multiplication         1
node.js                1
operating-system       1
oracle                 1
php                   13
php-5.3                1
php-7                  1
php-7.2                1
postgresql             1
python                 2
python-3.x             2
rest                   1
security               1
sftp                   1
syntax                 1
typeof                 1
undefined              1
unicode                1
utf-8                  1
dtype: int64

## the number of accepted answers against tags

In [657]:
def accepted_ans(user_id):

    accepted_tags = []
    pattern = '[<>]'

    regex = re.compile(pattern, flags = re.IGNORECASE)
    answers = Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]

    for i in range(len(answers)):
        tags = Posts.loc[(Posts.AcceptedAnswerId == answers.Id.iloc[i])]['Tags']
        try:
            accepted_tags.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_acanswers = pd.DataFrame({'Tag':accepted_tags}).groupby('Tag').size()
    return freq_acanswers

In [658]:
#example
accepted_ans(5734311)

Tag
angular       1
angular5      1
javascript    1
dtype: int64

## the number of comments against tags

In [453]:
Comments = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Comments.csv')

In [452]:
Comments

,Id,PostId,Score,Text,CreationDate,UserId
0,85065265,49030489,4,Possible duplicate of [Node.js - require empty...,2018-02-28T13:16:41.767,5734311.0
1,85065268,49030285,0,The problem with this is the bad structure log...,2018-02-28T13:16:45.110,2718972.0
2,85065269,49030414,0,Can you elaborate a little more on ViewModel? ...,2018-02-28T13:16:50.880,9305895.0
3,85065270,49022014,1,In `rollapply` the `width=` can be a one compo...,2018-02-28T13:16:51.247,516548.0
4,85065271,49029381,0,"Yeah, I'm testing here and it doesnt seem to b...",2018-02-28T13:16:52.050,1850609.0
5,85065272,49026068,1,Add some explanation to your answer,2018-02-28T13:16:51.907,2613552.0
6,85065273,49029230,0,yes thanks a lot man! this works just fine.,2018-02-28T13:16:55.800,8950119.0
7,85065274,49018387,0,Thanks @Lachezar Belev. I added the code abov...,2018-02-28T13:16:56.090,1026425.0
8,85065275,49027994,0,Thank you for your very clear answer :),2018-02-28T13:16:56.290,7065078.0
9,85065276,49025154,1,"*""The rest of the code can be found in our Git...",2018-02-28T13:17:00.447,418556.0


In [655]:
def comments_user(user_id):
    comments_tags = []
    pattern = '[<>]'


    regex = re.compile(pattern, flags = re.IGNORECASE)
    comments = Comments.loc[(Comments.UserId == user_id)]['PostId']

    for i in range(len(comments)):
        tags = Posts.loc[(Posts.Id == comments.iloc[i])]['Tags']
        try:
            comments_tags.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_comments = pd.DataFrame({'Tag':comments_tags}).groupby('Tag').size()
    return freq_comments

In [660]:
#example
comments_user(476)

Tag
.htaccess                8
ajax                     3
anaconda                 1
angular                  4
angularjs                1
api                      2
arrays                   3
autobahn                 1
calculation              1
character-encoding       2
chinese-locale           1
class                    3
css                      1
curl                     1
database                14
datetime                 4
dependency-injection     2
dictionary               4
dns                      1
ecmascript-6             3
ecomexpress              2
file                     1
file-upload              2
forms                    5
function                 2
generator                2
gravity-forms-plugin     1
html                    11
html5                    7
html5-video              1
                        ..
templates                1
timedelta                3
timezone                 6
transactions             1
twisted                  1
type-conversion         

## total number of activities against tags

In [786]:
def activities(user_id):
    answers = answers_user(user_id)
    acc_answers = accepted_ans(user_id)
    comments = comments_user(user_id)
    user_table = pd.DataFrame({'Answers' : answers ,'Accepted Answers' : acc_answers , 'Comments': comments})
    user_table = user_table.fillna(0)
    if len(user_table) != 0 :
        user_table['Total'] = user_table.apply(lambda row: (row['Answers'])+(row['Accepted Answers'])+(row['Comments']), axis=1)
        
    return user_table

In [798]:
#example
activities(205608)

,Accepted Answers,Answers,Comments,Total
activerecord,0.0,0.0,1.0,1.0
apache-poi,0.0,1.0,0.0,1.0
c#,0.0,0.0,1.0,1.0
database,1.0,3.0,4.0,8.0
duplicates,0.0,0.0,1.0,1.0
encoding,0.0,1.0,0.0,1.0
entity-framework,0.0,0.0,1.0,1.0
excel,0.0,0.0,1.0,1.0
excel-2016,0.0,0.0,1.0,1.0
excel-vba,0.0,0.0,1.0,1.0
